In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [8]:
# import required libraries
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [9]:
# configuring Chrome driver to use Chrome Browser
driver = webdriver.Chrome("C:/Users/v-msingarava/Desktop/chromedriver_win32/chromedriver")

# Site URL
url="https://www.boxofficemojo.com/year/world/?ref_=bo_nb_in_tab"
# driver.get(url)

In [10]:
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")

# to find number of tables in the page
table1 = soup.find_all("div", attrs={"class": "a-section imdb-scroll-table-inner"})
print("Number of tables on site: ",len(table1))

Number of tables on site:  1


In [15]:
# fetch table content using selenium
movie_data = []
n=1
for tr in soup.find_all('tr')[1:3]:
    n = n + 1
    movie_dict = {}
    tds = tr.find_all('td')
    movie_dict["Rank"] = tds[0].text
    movie_dict["Movie Name"] = tds[1].text
    movie_dict["Worldwide Revenue"] = tds[2].text
    
    movie_link = tds[1].findAll('a')
    movie_dict["First_level_link"] = "https://www.boxofficemojo.com" + movie_link[0]['href']
    
    driver.get("https://www.boxofficemojo.com" + movie_link[0]['href'])
    # First level Link
    # driver.find_element_by_xpath('//*[@id="table"]/div/table[2]/tbody/tr[{}]/td[2]/a'.format(n)).click()
    
    # movie page
    driver.find_element_by_xpath('//*[@id="title-summary-refiner"]/a').click()

    # movie info
    #divs = driver.find_elements_by_css_selector('#a-page > main > div > div.a-section.a-spacing-none.mojo-gutter.mojo-summary-table > div.a-section.a-spacing-none.mojo-summary-values.mojo-hidden-from-mobile')
    divs = driver.find_elements_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]')
    
    #divs = driver.find_elements_by_xpath('//div[@class="a-section a-spacing-none mojo-gutter mojo-summary-table"]')
    #div_count = 1
    #for div in divs:
    #    movie_dict["Others"] = div.text
        
    try:
        movie_dict[driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[1]/span[1]').text] = driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[1]/span[2]').text
        movie_dict[driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[2]/span[1]').text] = driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[2]/span[2]').text
        movie_dict[driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[3]/span[1]').text] = driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[3]/span[2]').text
        movie_dict[driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[4]/span[1]').text] = driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[4]/span[2]').text
        movie_dict[driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[5]/span[1]').text] = driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[5]/span[2]').text
        movie_dict[driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[6]/span[1]').text] = driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[6]/span[2]').text
        movie_dict[driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[7]/span[1]').text] = driver.find_element_by_xpath('//*[@id="a-page"]/main/div/div[3]/div[4]/div[7]/span[2]').text
    except Exception:
        pass
      
    movie_data.append(movie_dict)

    # print("Rank: {}, Movie Name: {}, Worldwide: {}".format(tds[0].text, tds[1].text, tds[2].text))

In [16]:
driver.close()

#headings = ["Rank", "Movie Name", "Worldwide Revenue", "First_level_link"]
# save the dataframe to local excel file
df = pd.DataFrame(data=movie_data)
df.head()
df.to_excel("C:/Users/v-msingarava/Desktop/movie_data_set.xlsx", index=False)